# CIFAR10_Test

使用CIFAR10数据集对CNN进行训练及测试

In [1]:
import tensorflow as tf
import numpy as np
import math
#import matplotlib.pyplot as plt
#%matplotlib inline
import time
import os
from mobile_net import MobileNet

/home/z_tomcato/assignment1/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
if sys.platform == "linux" :
    cifar10_dir = "../assignment2/assignment2/cs231n/datasets/cifar-10-batches-py"
else:
    cifar10_dir = 'cs231n/datasets'

from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    #cifar10_dir = 'cs231n/datasets'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


## Run Module

In [3]:

flags = tf.flags
"""
flags.DEFINE_integer('batch_size', 100, 'batch size')
flags.DEFINE_integer('num_epochs', 35, 'number of epochs')
flags.DEFINE_float('learning_rate', 0.04, 'init learning rate')
flags.DEFINE_float('dropout', 0.5, 'define dropout keep probability')
flags.DEFINE_float('max_grad_norm', 5.0, 'define maximum gradient normalize value')
flags.DEFINE_float('normalize_decay', 5.0, 'batch normalize decay rate')
flags.DEFINE_float('weight_decay', 0.0002, 'L2 regularizer weight decay rate')

flags.DEFINE_integer('print_every', 5, 'how often to print training status')
flags.DEFINE_string('name', None, 'name of result save dir')
"""
FLAGS = flags.FLAGS

In [4]:
  
def run_model(session, Xd, yd, Xv, yv, epochs=3, batch_size=100,print_every=10, learning_rate = 0.04, dropout = 0.5):
    print("Batch dataset initialized.\n# of training data: {}\n# of test data: {}\n# of class: {}"
          .format(Xd.shape[0], Xv.shape[0], 10))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)
    
    cnn_net = MobileNet(Xd.shape, 10)
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # tensorboard setting
        train_summary = tf.summary.merge([tf.summary.scalar("train_loss", cnn_net.loss),
                                          tf.summary.scalar("train_accuracy", cnn_net.accuracy)])
        test_summary = tf.summary.merge([tf.summary.scalar("val_loss", cnn_net.loss),
                                         tf.summary.scalar("val_accuracy", cnn_net.accuracy)])
        
        fileName = time.strftime("%Y%m%d_%H%M%S", time.localtime())
        fileName = os.path.normcase("./result/" + fileName)
        print(fileName)
        summary_writer = tf.summary.FileWriter(fileName, sess.graph)
        yd = yd.reshape([Xd.shape[0], 1])
        yv = yv.reshape([Xv.shape[0], 1])
        for current_epoch in range(epochs):
            # training step
            ###for x_batch, y_batch in batch_set.batches():
            print("#############################Epoch Start##############################")
            
            for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
                start = time.time()
                start_idx = (i*batch_size)%Xd.shape[0]
                idx = np.int32(train_indicies[start_idx:start_idx+batch_size])
                feed = {cnn_net.train_data:  Xd[idx,:, :, :], cnn_net.targets: yd[idx, :],
                        cnn_net.learning_rate: learning_rate, cnn_net.dropout: dropout, 
                        cnn_net.is_training : True}
                _, global_step, loss, accuracy, summary = \
                    sess.run([cnn_net.train_op, cnn_net.global_step, cnn_net.loss,
                              cnn_net.accuracy, train_summary], feed_dict=feed)
                summary_writer.add_summary(summary, global_step)
                if global_step % print_every == 0:
                    print("{}/{} ({} epochs) step, loss : {:.6f}, accuracy : {:.3f}, time/batch : {:.3f}sec"
                          .format(global_step, int(round(Xd.shape[0]/batch_size)) * epochs, current_epoch,
                                  loss, accuracy, time.time() - start))
            #验证集
            start, avg_loss, avg_accuracy = time.time(), 0, 0

            feed = {cnn_net.train_data: Xv,cnn_net.targets: yv,
                    cnn_net.learning_rate: learning_rate, cnn_net.dropout: 1.0, cnn_net.is_training : True}
            loss, accuracy, summary = sess.run([cnn_net.loss, cnn_net.accuracy, test_summary], feed_dict=feed)
            avg_loss = loss
            avg_accuracy = accuracy 
            summary_writer.add_summary(summary, current_epoch)
            print("{} epochs test result. loss : {:.6f}, accuracy : {:.3f}, time/batch : {:.3f}sec"
                  .format(current_epoch, avg_loss , avg_accuracy , time.time() - start))
            print("\n")
    return avg_loss,avg_accuracy      
    
   

In [10]:
tf.reset_default_graph()
with tf.Session() as sess:
    #with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
    sess.run(tf.global_variables_initializer())
    #print('Training')
    run_model(sess,X_train[:500],y_train[:500],X_val,y_val, epochs=6, batch_size=100,print_every=10, learning_rate = 0.001)

Batch dataset initialized.
# of training data: 500
# of test data: 1000
# of class: 10
./result/20180427_013049
#############################Epoch Start##############################
0 epochs test result. loss : 2.032577, accuracy : 0.247, time/batch : 18.479sec


#############################Epoch Start##############################
10/30 (1 epochs) step, loss : 1.370695, accuracy : 0.670, time/batch : 8.631sec
1 epochs test result. loss : 1.894448, accuracy : 0.309, time/batch : 18.368sec


#############################Epoch Start##############################
2 epochs test result. loss : 1.793587, accuracy : 0.352, time/batch : 18.800sec


#############################Epoch Start##############################
20/30 (3 epochs) step, loss : 0.285068, accuracy : 0.990, time/batch : 8.906sec
3 epochs test result. loss : 1.805395, accuracy : 0.343, time/batch : 18.971sec


#############################Epoch Start##############################
4 epochs test result. loss : 1.875831, accura